## KoBERT 모델 테스트

In [13]:
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# KoBERT 모델과 토크나이저 로드
model_name = "monologg/kobert"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# 텍스트 설정
text = """
[리딩 철강 브랜드의 변화]
국내 최초의 등대공장, 물류 fulfillment 등의 이슈를 접하며, POSCO에 대한 관심을 가졌습니다. 그 중 빠르게 디지털 전환을 성공시켜 smart-factory를 구축시킬 수 있었던 배경인 POSCO DX에 알게 되었습니다. 계획, 제선, 제강, 연주, 압연, 도금으로 이루어진 생산방식과 물류관리 자동화 방식은 리딩 철강 브랜드로 만들기에 충분했다고 생각합니다. 또한 글로벌 기업의 기술력이 아닌 사내 고유의 인프라를 활용해 신기술을 적용했기 때문에 확장성과 안정성을 동시에 갖출 수 있었습니다.
"""

# 문장을 개별적으로 나누기
sentences = [sentence.strip() for sentence in text.split('\n') if sentence.strip()]

# KoBERT로 각 문장의 임베딩 계산
embeddings = []
with torch.no_grad():
    for sentence in sentences:
        inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True).to(torch.device("cpu"))
        outputs = model(**inputs)
        cls_embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        embeddings.append(cls_embedding.flatten())

# 코사인 유사도로 중심 문장 찾기
embeddings = np.array(embeddings)
similarity_matrix = cosine_similarity(embeddings)
sentence_scores = similarity_matrix.mean(axis=1)
top_sentence_index = np.argmax(sentence_scores)
headline_sentence = sentences[top_sentence_index]

# 간단한 키워드 추출 및 재구성
keywords = ["POSCO DX", "디지털 전환", "스마트 팩토리"]
headline = " • " + ", ".join(keywords) + "에 대한 도전과 혁신"
print("한줄 요약 (개조식):", headline)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


한줄 요약 (개조식):  • POSCO DX, 디지털 전환, 스마트 팩토리에 대한 도전과 혁신
